In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score
from sklearn.preprocessing import PolynomialFeatures,LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('loan_approval_dataset.csv')
df

,Age,Salary,Credit_Score,Loan_Amount,Loan_Term,Employment_Status,Residence_Type,Previous_Default,Loan_Approved
0,56,136748,584,38209,36 months,Employed,Owned,Yes,Yes
1,46,25287,815,27424,24 months,Self-Employed,Rented,No,Yes
2,32,146593,398,42396,12 months,Unemployed,Rented,Yes,Yes
3,60,54387,696,11370,24 months,Unemployed,Owned,No,No
4,25,28512,788,14528,12 months,Employed,Owned,No,No
...,...,...,...,...,...,...,...,...,...
995,22,49241,500,41020,24 months,Self-Employed,Owned,No,Yes
996,40,116214,423,12415,48 months,Self-Employed,Owned,No,Yes
997,27,64569,300,28155,36 months,Self-Employed,Rented,Yes,Yes
998,61,31745,490,48884,12 months,Self-Employed,Mortgage,No,Yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Age                1000 non-null   int64 
 1   Salary             1000 non-null   int64 
 2   Credit_Score       1000 non-null   int64 
 3   Loan_Amount        1000 non-null   int64 
 4   Loan_Term          1000 non-null   object
 5   Employment_Status  1000 non-null   object
 6   Residence_Type     1000 non-null   object
 7   Previous_Default   1000 non-null   object
 8   Loan_Approved      1000 non-null   object
dtypes: int64(4), object(5)
memory usage: 70.4+ KB


In [4]:
df.drop("Loan_Term",axis=1,inplace=True)

In [5]:
X=df.drop('Loan_Approved',axis=1)
y=df['Loan_Approved']

In [6]:
cat_col=X.select_dtypes(include='object')
encode=OneHotEncoder(sparse_output=False)
encodex=encode.fit_transform(cat_col)
encodex
cat_df=pd.DataFrame(encodex,columns=encode.get_feature_names_out())

In [7]:
num_col=X.select_dtypes(exclude='object')
scaler=StandardScaler()
val=scaler.fit_transform(num_col)
val
num_df=pd.DataFrame(val,columns=scaler.get_feature_names_out())

In [8]:
updateX=pd.concat([cat_df,num_df],axis=1)
updateX

,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Residence_Type_Mortgage,Residence_Type_Owned,Residence_Type_Rented,Previous_Default_No,Previous_Default_Yes,Age,Salary,Credit_Score,Loan_Amount
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.112882,1.353517,0.048655,0.861782
1,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.371652,-1.572999,1.513836,0.020448
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,-0.666069,1.612007,-1.131101,1.188408
3,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.409373,-0.808950,0.759046,-1.231919
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.184929,-1.488324,1.342581,-0.985565
...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,-1.407298,-0.944064,-0.484138,1.081067
996,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,-0.073085,0.814377,-0.972531,-1.150399
997,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.036683,-0.541612,-1.752692,0.077473
998,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.483496,-1.403438,-0.547565,1.694535


In [9]:
Xtrain,Xtest,ytrain,ytest=train_test_split(updateX,y,train_size=0.8,random_state=42)

In [10]:
model=LogisticRegression()
model.fit(Xtrain,ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [11]:
model.score(Xtest,ytest)

0.525

In [12]:
new_input = pd.DataFrame({'Age':[22,30],'Salary':[15000,35000],'Credit_Score':[650,780],'Loan_Amount':[150000,80000],'Employment_Status':['Self-Employed','Employed'],'Residence_Type':['Mortgage','Rented'],'Previous_Default':['Yes','No']})
new_input

,Age,Salary,Credit_Score,Loan_Amount,Employment_Status,Residence_Type,Previous_Default
0,22,15000,650,150000,Self-Employed,Mortgage,Yes
1,30,35000,780,80000,Employed,Rented,No


In [13]:
cat_col=new_input.select_dtypes(include='object')
encodenewX=encode.transform(cat_col)

cat_df=pd.DataFrame(encodenewX,columns=encode.get_feature_names_out())
cat_df

,Employment_Status_Employed,Employment_Status_Self-Employed,Employment_Status_Unemployed,Residence_Type_Mortgage,Residence_Type_Owned,Residence_Type_Rented,Previous_Default_No,Previous_Default_Yes
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [14]:
num_col=new_input.select_dtypes(exclude='object')
val=scaler.transform(num_col)
num_df=pd.DataFrame(val,columns=scaler.get_feature_names_out())

In [15]:
updatedX=pd.concat([cat_df,num_df],axis=1)

In [16]:
model.predict(updatedX)

array(['Yes', 'Yes'], dtype=object)

# PIPE LINE AND COLUMN TRANSFORMER

In [17]:
# pipeline = Pipeline(steps=[
#   ('name of the process',model),
#    ('name of the process',model),
#    .......
#    ....
#    ....
# ])
# '''

# '''
# preprocessor = ColumnTransformer(
#     transformer=[
#         ('name',preprocessing,col_names),
#         ('name1',preprocessing1,col_names1)
#     ]
# )
# '''

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


df=pd.read_csv('loan_approval_dataset.csv')
df.drop('Loan_Term',axis=1,inplace=True)
X=df.drop('Loan_Approved',axis=1)
y=df['Loan_Approved']


Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,train_size=0.8,random_state=42)
cat_cols=X.select_dtypes(include='object').columns
num_cols=X.select_dtypes(include=['int','float64']).columns


preprocessing=ColumnTransformer(
              transformers=[
                  ('standardscaler',StandardScaler(),num_cols),
                  ('OneHotEncoder',OneHotEncoder(),cat_cols) ])


pipeline=Pipeline(
         steps=[('preprocessing',preprocessing),
                ('logistic',LogisticRegression())])



pipeline.fit(Xtrain,ytrain)
# Xtrain

,steps,"[('preprocessing', ...), ('logistic', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('standardscaler', ...), ('OneHotEncoder', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
pipeline.predict(Xtest)

In [20]:
pipeline.score(Xtest,ytest)

0.525

In [21]:
new_input = pd.DataFrame({'Age':[22,30],'Salary':[15000,35000],'Credit_Score':[650,780],'Loan_Amount':[150000,80000],'Employment_Status':['Self-Employed','Employed'],'Residence_Type':['Mortgage','Rented'],'Previous_Default':['Yes','No']})
new_input

,Age,Salary,Credit_Score,Loan_Amount,Employment_Status,Residence_Type,Previous_Default
0,22,15000,650,150000,Self-Employed,Mortgage,Yes
1,30,35000,780,80000,Employed,Rented,No


In [22]:
pipeline.predict(new_input)

array(['Yes', 'Yes'], dtype=object)